# Full flexibility for synaptic parameter sharing

In this tutorial, you will learn how to:

- share parameters of synapses  
- remove any kind of synaptic parameter sharing

Here is a code snippet which you will learn to understand in this tutorial:
```python
net = ...  # See tutorial on Basics of Jaxley.

# The same parameter for all synapses
net.make_trainable("Ionotropic_gS")

# An individual parameter for every synapse.
net.select(edges="all").make_trainable("Ionotropic_gS")

# Share synaptic conductances emerging from the same neurons.
sub_net = net.select(edges=[0, 1, 2])
sub_net.edges["controlled_by_param"] = sub_net.edges["global_pre_cell_index"]
sub_net.make_trainable("Ionotropic_gS")
```

In a [previous tutorial](https://jaxleyverse.github.io/jaxley/latest/tutorial/03_setting_parameters/) about training networks, we briefly touched on parameter sharing. In this tutorial, we will show you how you can flexibly share parameters within a network.

In [68]:
import jaxley as jx
from jaxley.channels import Na, K, Leak
from jaxley.connect import fully_connect
from jaxley.synapses import IonotropicSynapse

### Preface: Building the network

We first build a network consisting of six neurons, in the same way as we showed in the previous tutorials:

In [69]:
dt = 0.025
t_max = 10.0

comp = jx.Compartment()
branch = jx.Branch(comp, nseg=2)
cell = jx.Cell(branch, parents=[-1, 0])
net = jx.Network([cell for _ in range(6)])
fully_connect(net.cell([0, 1, 2]), net.cell([3, 4, 5]), IonotropicSynapse())

### Sharing parameters by modifying `controlled_by_param`

In [70]:
df = net.edges
df = df.query("global_pre_cell_index in [0, 1, 2]")
subnetwork = net.select(edges=df.index)

df = subnetwork.edges
df["controlled_by_param"] = df["global_pre_cell_index"]
subnetwork.make_trainable("IonotropicSynapse_gS")

Number of newly added trainable parameters: 3. Total number of trainable parameters: 3


Let's look at this line by line. First, we exactly follow the previous tutorial in selecting the synapses which we are interested in training (i.e., the ones whose presynaptic neuron has index 0, 1, 2):

In [71]:
df = net.edges
df = df.query("global_pre_cell_index in [0, 1, 2]")
subnetwork = net.select(edges=df.index)

As second step, we enable parameter sharing. This is done by setting the `controlled_by_param`. Synapses that have the same value in `controlled_by_param` will be shared. Let's inspect `controlled_by_param` _before_ we modify it:

In [72]:
subnetwork.edges[["global_pre_cell_index", "controlled_by_param"]]

,global_pre_cell_index,controlled_by_param
0,0,0
1,0,1
2,0,2
3,1,3
4,1,4
5,1,5
6,2,6
7,2,7
8,2,8


Every synapse has a different value. Because of this, no synaptic parameters will be shared. To enable parameter sharing we override the `controlled_by_param` column with the presynaptic cell index:

In [73]:
df = subnetwork.edges
df["controlled_by_param"] = df["global_pre_cell_index"]

In [74]:
df[["global_pre_cell_index", "controlled_by_param"]]

,global_pre_cell_index,controlled_by_param
0,0,0
1,0,0
2,0,0
3,1,1
4,1,1
5,1,1
6,2,2
7,2,2
8,2,2


Now, all we have to do is to make these synaptic parameters trainable with the `make_trainable()` method:

In [75]:
subnetwork.make_trainable("IonotropicSynapse_gS")

Number of newly added trainable parameters: 3. Total number of trainable parameters: 6


It correctly says that we added three parameters (because we have three cells, and we share individual synaptic parameters).

### A more involved example: sharing by pre- and post-synaptic cell type

As a example, consider the following: We have a fully connected network of six cells. Each cell falls into one of three cell types:

In [133]:
net = jx.Network([cell for _ in range(6)])
fully_connect(net.cell("all"), net.cell("all"), IonotropicSynapse())

net.cell([0, 1]).add_to_group("exc")
net.cell([2, 3]).add_to_group("inh")
net.cell([4, 5]).add_to_group("unknown")

We want to make all synapses that start from excitatory or inhibitory neurons trainable. In addition, we want to use the same parameter for synapses if they have the same pre- **and** post-synaptic cell type.

Let's first query all synapses that start from excitatory or inhibitory neurons:

In [134]:
exc_cell_inds = net.exc.nodes["global_cell_index"].unique().tolist()
inh_cell_inds = net.inh.nodes["global_cell_index"].unique().tolist()

In [135]:
df = net.edges
df = df.query(f"global_pre_cell_index in {exc_cell_inds + inh_cell_inds}")
print(f"There are {len(df)} synapses to be changed.")

subnetwork = net.select(edges=df.index)

There are 24 synapses to be changed.


Next, we deal with parameter sharing. As described above, we want to use the same parameter only if neurons have the same pre- and post-synaptic cell **type**.

In [136]:
exc_inds = net.exc.nodes.index.to_numpy().tolist()

In [138]:
net.nodes["exc"] = False
net.nodes["exc"].loc[exc_inds] = True

/var/folders/kv/39t9blx569l5_sxxzx89vb3c0000gn/T/ipykernel_76534/1607855332.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  net.nodes["exc"].loc[exc_inds] = True
/var/folders/kv/39t9blx569l5_sxxzx89vb3c0000gn/T/ipykernel_76534/1607855332

In [139]:
net.nodes

,global_cell_index,global_branch_index,global_comp_index,local_cell_index,local_branch_index,local_comp_index,length,radius,axial_resistivity,capacitance,v,controlled_by_param,exc
0,0,0,0,0,0,0,10.0,1.0,5000.0,1.0,-70.0,0,True
1,0,0,1,0,0,1,10.0,1.0,5000.0,1.0,-70.0,0,True
2,0,1,2,0,1,0,10.0,1.0,5000.0,1.0,-70.0,0,True
3,0,1,3,0,1,1,10.0,1.0,5000.0,1.0,-70.0,0,True
4,1,2,4,1,0,0,10.0,1.0,5000.0,1.0,-70.0,0,True
5,1,2,5,1,0,1,10.0,1.0,5000.0,1.0,-70.0,0,True
6,1,3,6,1,1,0,10.0,1.0,5000.0,1.0,-70.0,0,True
7,1,3,7,1,1,1,10.0,1.0,5000.0,1.0,-70.0,0,True
8,2,4,8,2,0,0,10.0,1.0,5000.0,1.0,-70.0,0,False
9,2,4,9,2,0,1,10.0,1.0,5000.0,1.0,-70.0,0,False


In [129]:
net.nodes

,global_cell_index,global_branch_index,global_comp_index,local_cell_index,local_branch_index,local_comp_index,length,radius,axial_resistivity,capacitance,...,controlled_by_param,exc,0,1,2,3,4,5,6,7
0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,1.0,5000.0,1.0,...,0.0,False,b'',b'',b'',b'',b'',b'',b'',b''
1,0.0,0.0,1.0,0.0,0.0,1.0,10.0,1.0,5000.0,1.0,...,0.0,False,b'',b'',b'',b'',b'',b'',b'',b''
2,0.0,1.0,2.0,0.0,1.0,0.0,10.0,1.0,5000.0,1.0,...,0.0,False,b'',b'',b'',b'',b'',b'',b'',b''
3,0.0,1.0,3.0,0.0,1.0,1.0,10.0,1.0,5000.0,1.0,...,0.0,False,b'',b'',b'',b'',b'',b'',b'',b''
4,1.0,2.0,4.0,1.0,0.0,0.0,10.0,1.0,5000.0,1.0,...,0.0,False,b'',b'',b'',b'',b'',b'',b'',b''
5,1.0,2.0,5.0,1.0,0.0,1.0,10.0,1.0,5000.0,1.0,...,0.0,False,b'',b'',b'',b'',b'',b'',b'',b''
6,1.0,3.0,6.0,1.0,1.0,0.0,10.0,1.0,5000.0,1.0,...,0.0,False,b'',b'',b'',b'',b'',b'',b'',b''
7,1.0,3.0,7.0,1.0,1.0,1.0,10.0,1.0,5000.0,1.0,...,0.0,False,b'',b'',b'',b'',b'',b'',b'',b''
8,2.0,4.0,8.0,2.0,0.0,0.0,10.0,1.0,5000.0,1.0,...,0.0,False,b'',b'',b'',b'',b'',b'',b'',b''
9,2.0,4.0,9.0,2.0,0.0,1.0,10.0,1.0,5000.0,1.0,...,0.0,False,b'',b'',b'',b'',b'',b'',b'',b''


In [91]:
mapping = net.nodes

In [92]:
mapping

,global_cell_index,global_branch_index,global_comp_index,local_cell_index,local_branch_index,local_comp_index,length,radius,axial_resistivity,capacitance,v,controlled_by_param
0,0,0,0,0,0,0,10.0,1.0,5000.0,1.0,-70.0,0
1,0,0,1,0,0,1,10.0,1.0,5000.0,1.0,-70.0,0
2,0,1,2,0,1,0,10.0,1.0,5000.0,1.0,-70.0,0
3,0,1,3,0,1,1,10.0,1.0,5000.0,1.0,-70.0,0
4,1,2,4,1,0,0,10.0,1.0,5000.0,1.0,-70.0,0
5,1,2,5,1,0,1,10.0,1.0,5000.0,1.0,-70.0,0
6,1,3,6,1,1,0,10.0,1.0,5000.0,1.0,-70.0,0
7,1,3,7,1,1,1,10.0,1.0,5000.0,1.0,-70.0,0
8,2,4,8,2,0,0,10.0,1.0,5000.0,1.0,-70.0,0
9,2,4,9,2,0,1,10.0,1.0,5000.0,1.0,-70.0,0


# TODO: we need access to the cell type!

In [41]:
df = subnetwork.edges
df["presynaptic_sype"] = 
df["controlled_by_param"] = df.groupby(["global_pre_cell_index", "global_post_cell_index"]).ngroup()
subnetwork.make_trainable("IonotropicSynapse_gS")

Number of newly added trainable parameters: 9. Total number of trainable parameters: 33


### Summary

In this tutorial, you learned how you can flexibly share synaptic parameters. This works by first using `select()` to identify which synapses to make trainable, and by then modifying `controlled_by_param` to customize parameter sharing.